In [21]:
# 라이브러리 호출
import os
import numpy as np
from PIL import Image
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, MiniBatchKMeans, BisectingKMeans, SpectralClustering
import matplotlib.pyplot as plt

In [3]:
# # 이미지 리 사이징 하기

# dataset_dir = "./dataset/"
# folders = os.listdir(dataset_dir)
# folders = [f for f in folders if not f.startswith('.')]

# for d in folders:
#     target_dir = dataset_dir + d
#     print(target_dir)
    
#     format = [".jpg", ".png", ".jpeg", "bmp", ".JPG", ".PNG", "JPEG", "BMP"]  # 지원하는 파일 형태의 확장자들
#     for (path, dirs, files) in os.walk(target_dir):
#         for file in files:
#             if file.endswith(tuple(format)):
#                 image = Image.open(path + "/" + file)
#                 # print(image.filename)
#                 # print(image.size)

#                 if image.mode != 'RGB':
#                     image = image.convert('RGB')

#                 image = image.resize((100,100))
#                 if os.path.exists(f"./converted_dataset/{d}/") == False:
#                     os.makedirs(f"./converted_dataset/{d}/")

#                 image.save(f"./converted_dataset/{d}/{file}")
#                 # print(image.size)

#             else:
#                 print(path)
#                 print("InValid", file)

#     # 변환할 이미지 목록 불러오기
#     image_path = f"./converted_dataset/{d}/"

#     img_list = os.listdir(image_path)  # 디렉토리 내 모든 파일 불러오기

#     img_list_np = []
#     for i in img_list:
#         img = Image.open(image_path + i)
#         img_array = np.array(img)
#         img_list_np.append(img_array)
#         print(i, " 추가 완료 - 구조:", img_array.shape)  # 불러온 이미지의 차원 확인 (세로X가로X색)
#         # print(img_array.T.shape) #축변경 (색X가로X세로)

#     img_np = np.array(img_list_np)  # 리스트를 numpy로 변환

#     if os.path.exists(f"./np_converted_dataset/") == False:
#         os.makedirs(f"./np_converted_dataset/")

#     np.save(f'./np_converted_dataset/{d}', img_np)  
#     print(img_np.shape) # (파일수 * 가로 * 세로 * RGB)
#     print("결과: 정상적 으로 저장 완료")

In [4]:
def draw_sample_data(arr, ratio=1):
    n = len(arr)  # n은 샘플 개수입니다
    # 한 줄에 10개씩 이미지를 그립니다. 샘플 개수를 10으로 나누어 전체 행 개수를 계산합니다.
    # print(n)
    rows = int(np.ceil(n / 10))
    # 행이 1개 이면 열 개수는 샘플 개수입니다. 그렇지 않으면 10개입니다.
    cols = n if rows < 2 else 10
    fig, axs = plt.subplots(rows, cols,
                            figsize=(cols * ratio, rows * ratio), squeeze=False)
    for i in range(rows):
        # print(i)
        for j in range(cols):
            if i * 10 + j < n:  # n 개까지만 그립니다.
                # print(i)
                axs[i, j].imshow(arr[i * 10 + j], cmap='gray_r')
            axs[i, j].axis('off')
    plt.show()

In [43]:
np_converted_dir = f'./np_converted_dataset/'
# many_real_dir = ['L2_3.npy', 'L2_12.npy', 'L2_24.npy','L2_41.npy','L2_50.npy']
np_converted_dir_list = [f for f in os.listdir(np_converted_dir) if f != '.DS_Store']

arr0 = np.load(np_converted_dir+np_converted_dir_list[0])
arr0_2d = arr0.reshape(-1, 100 * 100 * 3)

for file_ in np_converted_dir_list[1:]:
    np_data = np.load(np_converted_dir+file_)
    np_data_2d = np_data.reshape(-1, 100 * 100 * 3)
    arr0_2d = np.concatenate((arr0_2d, np_data_2d))
    
    # print("sample_data.shape:", np_data.shape)  
    # print("sample_data_2d.shape:", np_data_2d.shape)

print(arr0_2d.shape)    
clustering = KMeans(n_clusters=2, random_state = 2022)  # random_state=42 -> 항상 랜덤값이 같게 하기위해 쓴다.
reduced_X = PCA(n_components=30).fit(arr0_2d)
clustering.fit(reduced_X)

# label = file_.split('.')[0]
# print(f'Label: {label}')
print(f'총 이미지 수: {len(np_data)}')
# print(f'일러스트 이미지 수: {len(np_data[clustering.labels_ == 0])}')
# print(f'실사 이미지 수: {len(np_data[clustering.labels_ == 1])}')

draw_sample_data(np_data[clustering.labels_ == 0])
draw_sample_data(np_data[clustering.labels_ == 1])
# draw_sample_data(np_data[clustering.labels_ == 2])
    

(25603, 30000)
